In [ ]:
import json
import copy
import time
import matplotlib.pyplot as plt
from matplotlib import colors

# Define the colour map used by ARC tasks (0–9)
cmap = colors.ListedColormap([
    '#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
    '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'
])
norm = colors.Normalize(vmin=0, vmax=9)


def plot_grid(grid, ax=None, title=None, show_grid=True):
    """Visualize a single grid using the ARC colour scheme."""
    if ax is None:
        fig, ax = plt.subplots()
    ax.imshow(grid, cmap=cmap, norm=norm, interpolation='nearest')
    h,w=len(grid),len(grid[0])
    ax.set_xticks([])
    ax.set_yticks([])
    if show_grid:
        # 细网格: 使用minor ticks形成0.5偏移的格线，避免依赖numpy
        xt=[i-0.5 for i in range(w+1)]
        yt=[i-0.5 for i in range(h+1)]
        ax.set_xticks(xt, minor=True)
        ax.set_yticks(yt, minor=True)
        ax.grid(which='minor', linewidth=0.3, color='white', alpha=0.6)
    if title:
        ax.set_title(title)
    return ax


def load_task(task_id):
    """Load a task JSON given its numeric ID.

    Expects a file named like `task117.json` in the current directory."""
    path = f"/kaggle/input/google-code-golf-2025/task{int(task_id):03d}.json"
    with open(path, 'r', encoding='utf-8') as f:
        task = json.load(f)
    return task

In [ ]:
HINTS={}

In [ ]:
HINTS[400]="这是一个模式提取和转换的任务，输入是一个24x24的网格，其中包含一个由颜色1标记的5x5区域，你需要找到这个标记区域的位置，然后提取以该位置为中心的5x5区域，并对该区域进行反转操作得到输出。"
HINTS[399]="这是一个空间映射任务，输入和输出的特征是输入包含多个2x2的2色块分布在网格中，输出是3x3的网格，需要识别输入中2x2色块的位置分布，并将其映射到输出网格的对应位置标记为1。"
HINTS[398]="这是一个矩阵生成任务，输入是1x5的向量，输出是z×z的方阵，其中z=25-5×(输入中0的数量)。你需要将输入向量去除首尾的0后，放置在输出矩阵的反对角线上，每行向右偏移相应的位置。"
HINTS[397]="这是一个模式标记任务，输入和输出的特征是识别输入网格中的特定颜色模式（如相邻的相同颜色对或小结构），然后在检测到的模式周围或下方添加2x2的颜色3块作为标记。"
HINTS[396]="这是一个对象轮廓提取任务，输入包含一个由非零颜色组成的矩形对象和零背景，你需要识别最大的连续非零矩形区域，然后裁剪该区域并生成其边框轮廓，内部区域用零填充，边框使用输入中最频繁的非零颜色。"
HINTS[395]="这是一个网格比较任务，输入是6x3的网格，输出是3x3的网格，你需要将输入分成上下两个3x3部分，然后逐列比较对应位置，如果两个位置都是0，则输出2，否则输出0。"
HINTS[393]="这是一个颜色频率统计任务，输入是包含多个颜色区域的网格，输出是3x1的矩阵，需要识别所有非零颜色并按出现频率从低到高排序，取频率最低的3个颜色作为输出结果。"
HINTS[392]="这是一个几何形状扩展任务，输入包含特定颜色的几何形状，你需要识别这些形状的边界，然后用颜色5和原始颜色创建交替的条纹模式填充整个网格。"
HINTS[394]="这是一个网格提取和合并的任务，输入网格包含背景色0和非零物体，输出是较小的网格，其尺寸由包含第一个背景0的行中背景0的数量决定，然后从输入中提取两个偏移量的块，并通过按位或操作合并它们。"
HINTS[391]="这是一个频率统计任务，输入是由0分隔的2x2颜色块组成的网格，你需要统计所有非零颜色的出现频率，然后提取出现次数最少的3种颜色，按频率从低到高排列作为输出。"
HINTS[390]="这是一个对称反射任务，输入包含由颜色2组成的闭合形状和内部颜色5的点，你需要识别非零点边界框的中心轴，然后将颜色5的点沿着边界框较长轴的方向进行对称反射。"
HINTS[389]="这是一个颜色替换任务，输入图中包含颜色5和其他颜色，你需要识别所有颜色为5的像素，然后将它们替换为0，其他颜色保持不变。"
HINTS[388]="这是一个网格放大的任务，输入和输出的特征是尺寸放大2倍，你需要识别输入中每列的非零元素，然后将输出中对应列的原始零值位置替换为颜色8，其他非零颜色保持不变。"
HINTS[387]="这是一个对称矩形生成任务，输入的特征是网格中有两个对称分布的非零点，你需要以这两个点为对角点确定矩形区域，在矩形边界上创建3×3的交替色块模式，并在对角线方向用颜色5填充连接线。"
HINTS[386]="这是一个网格转换任务，输入是4x7网格，其中第3列总是颜色1，输出是4x3网格，需要移除中间的1列，然后将前3列和后3列通过某种逻辑运算（如按位操作或条件判断）组合成输出，其中颜色3的出现与特定条件相关。"
HINTS[385]="这是一个矩阵变换任务，输入和输出的尺寸相同，你需要识别输入矩阵底部连续的非零行区域，然后将这些行移动到矩阵顶部，保持其他行的相对顺序不变。"
HINTS[384]="这是一个形状提取和扩展的任务，输入是稀疏的4像素点阵，你需要识别这些点形成的几何结构（通常是矩形轮廓），然后生成一个填充的、尺寸更大的规则矩形输出，其中4像素形成实心块，0像素作为背景。"
HINTS[383]="这是一个形状边界处理任务，输入包含一个由主要颜色构成的矩形区域，内部有一个由次要颜色构成的形状，你需要识别内部形状的边界，将内部形状的部分区域替换为主要颜色，同时在外部区域的对称位置添加次要颜色的标记点。"
HINTS[382]="这是一个模式传播任务，输入中的8颜色点作为种子，需要识别这些种子的位置，然后根据网格结构将8颜色向右下方向传播扩展，形成连续的8颜色区域，同时保持2颜色点不变。"
HINTS[381]="这是一个行内填充任务，输入包含背景色0和对象色2，你需要识别每行中被2包围的0区域，并将这些0填充为9。"
HINTS[380]="这是一个矩阵旋转任务，输入和输出的特征是3x3网格且颜色分布相同，你需要识别输入矩阵的图案，然后执行逆时针旋转90度的操作。"
HINTS[378]="这是一个边界框填充任务，输入和输出的特征是保持相同尺寸，你需要识别输入中所有非零像素形成的边界框的四个角点，找到角点中出现次数最少的非零颜色，然后从四个角点沿对角线方向向内填充该颜色。"
HINTS[379]="这是一个生长传播任务，输入包含背景(0)、线条(8)和种子点(2)，你需要识别种子点(2)的位置，让它们沿着垂直方向生长，当遇到线条(8)时，在线条周围创建3x3的8色方块，并将种子点穿过线条继续传播。"
HINTS[377]="这是一个边界提取任务，输入是包含多个嵌套颜色矩形的大图像，输出是提取最内层几层矩形边界的小图像，需要识别颜色矩形的嵌套层次结构，然后提取并压缩最内层的边界信息。"
HINTS[375]="这是一个对角线清零的任务，输入是一个奇数尺寸的正方形矩阵，中心位置为0，其他位置为相同颜色，你需要将主对角线和反对角线上的所有元素设为0，其他位置保持不变。"
HINTS[374]="这是一个连通分量分类任务，输入和输出的特征是识别颜色5的连通分量，根据其方向（垂直或水平）和相对位置（最长垂直分量或其他）将其分别映射为颜色1、4或2。"
HINTS[373]="这是一个排列变换任务，输入是2x6的网格，每行由单一颜色组成，你需要将两行的颜色按列交替排列，第一行输出为[行1色,行2色,行1色,行2色,...]，第二行输出为相反的顺序[行2色,行1色,行2色,行1色,...]"
HINTS[376]="这是一个矩阵扩展任务，输入是一个n行17列的矩阵，输出是一个4n-3行17列的矩阵，你需要将输入矩阵垂直扩展，生成序列为[原始行0至n-1, 反向行n-2至0, 正向行1至n-1, 反向行n-2至0]的新矩阵"
HINTS[371]="这是一个标记扩展任务，输入图中包含颜色为1的特殊点，需要在每个特殊点的上、下、左、右四个相邻位置（形成十字形）将颜色0改为颜色3，而特殊点本身保持不变。"
HINTS[370]="这是一个方向传播任务，输入图中有一个特殊颜色的点，你需要识别该点的位置，找到距离最远的另一个特殊点确定传播方向，然后沿着该方向将特殊颜色传播到整个网格，但遇到障碍物（颜色0）时停止传播。"
HINTS[369]="这是一个连通区域大小标记任务，输入包含背景色5和标记色0，你需要识别所有连通区域的大小，将大小为1的区域标记为3，大小为2的区域标记为2，大小为3的区域标记为1，保持背景色5不变。"
HINTS[372]="这是一个行选择任务，输入的特征是中间有一行全为5的颜色作为分隔符，输出的特征是从输入中移除分隔行后选择特定的5行重新排列组成输出网格，需要识别分隔行位置然后执行行选择操作。"
HINTS[368]="这是一个模式替换任务，输入中有颜色5的区域和其他非零颜色区域，你需要从非5区域提取模式模板，然后用这个模板替换所有颜色5的区域，保持网格尺寸不变。"
HINTS[367]="这是一个区域填充任务，输入是由颜色5形成的边界和颜色0的背景，你需要识别由颜色5组成的完整闭合矩形边界，然后将这些矩形内部的所有颜色0像素填充为颜色4。"
HINTS[366]="这是一个图像合成任务，输入图像被分为模板区和内容区，你需要识别模板区中的非背景图案，在内容区中找到匹配位置，然后将模板图案复制到内容区的对应位置，生成输出图像。"
HINTS[365]="这是一个目标区域提取任务，输入包含多个连通区域，你需要识别包含最多颜色2的连通区域，然后提取该区域的边界框作为输出，去除背景色0。"
HINTS[364]="这是一个多边形分类任务，输入是黑色背景上的青色(3)多边形，你需要识别每个连通区域的多边形形状复杂度，根据其边角特征将简单形状着色为红色(1)，复杂形状着色为绿色(2)，有直角的多边形着色为青色(6)。"
HINTS[363]="这是一个模式填充任务，输入和输出的特征是包含颜色0、5、2的网格，你需要识别颜色2组成的特定模式，然后在网格中找到可以放置该模式的空白区域（颜色0的位置），将模式复制到这些区域并用颜色2填充。"
HINTS[362]="这是一个行移动任务，输入中包含一个完整的单色行和若干颜色为5的标记点，你需要根据颜色5的位置计算出完整行应该向下移动的行数，然后将完整行移动到新位置，同时移除所有颜色为5的像素点。"
HINTS[361]="这是一个对称扩展任务，输入和输出的特征是包含一个或多个对称中心点的图案，你需要识别输入图案的对称中心和对称轴，然后基于镜像或旋转对称规则在对称位置复制和扩展图案元素。"
HINTS[359]="这是一个模式统一任务，输入和输出的尺寸相同，你需要比较行和列的颜色多样性，选择颜色种类总数更多的方向，然后将该方向的每个单元（行或列）替换为其中出现最频繁的颜色。"
HINTS[360]="这是一个对称最大值任务，输入矩阵有一个固定的中心列（颜色5），你需要将每一行的左侧4列和右侧4列（对称位置）进行逐对最大值操作，生成4列的输出矩阵。"
HINTS[358]="这是一个模式扩展任务，输入和输出的特征是输入图中包含一个非零颜色的种子序列，你需要识别该种子序列的位置和内容，然后沿着水平或垂直方向周期性地重复该序列来填充整个输出图。"
HINTS[357]="这是一个生成反弹模式的任务，输入是一个几乎全0的矩阵（仅最后一行有一个1），输出是相同尺寸的矩阵，其中每行有一个1，其余为8，且1的位置沿着反对角线方向每行移动一步，并在边界处反弹。"
HINTS[356]="这是一个连接任务，输入和输出的特征是包含颜色8的稀疏点阵，你需要识别每行和每列中颜色8的最左/最右和最上/最下位置，然后在行方向上连接最左和最右的8点，在列方向上连接最上和最下的8点，形成水平和垂直的线段。"
HINTS[354]="这是一个颜色替换任务，输入和输出的特征是包含颜色5的块状区域，你需要识别每行中第一个非零非5的颜色，然后将该行中的所有颜色5替换为这个颜色。"
HINTS[355]="这是一个区域分割和异常检测的任务，输入网格被分为两个主要颜色区域且包含少量异常点，你需要识别两个区域的分界线，找到包含异常点较少的区域，然后输出该区域的主要颜色。"
HINTS[353]="这是一个位置调整任务，输入图中有一个3和一个4，你需要将3向4的方向移动一步（沿行和列方向分别靠近），但保持与4的相对位置关系，4的位置保持不变。"
HINTS[352]="这是一个基于目标像素的膨胀任务，输入图中包含颜色2的像素点，你需要识别所有颜色2像素的位置，然后将每个颜色2像素周围3×3区域内的0像素填充为颜色1，其他像素保持不变。"
HINTS[351]="这是一个子网格提取任务，输入是16x16网格且包含连续的3值区域，你需要识别3值区域的位置，然后从输入网格中提取一个特定的5x5子网格，该子网格的位置与3值区域的分布相关。"
HINTS[350]="这是一个连接任务，输入和输出的特征是识别颜色为1的点作为关键节点，然后在行和列方向上用颜色8填充这些关键节点之间的所有0值位置，形成连通路径。"
HINTS[349]="这是一个对象装饰任务，输入中只有0和9两种颜色，其中9形成矩形块。你需要识别所有的9矩形块，为每个矩形块添加颜色3的边框，并在某些区域添加颜色1的填充，同时保持原始9矩形块不变"
HINTS[348]="这是一个对角线填充任务，输入特征是一条垂直的颜色7线条，输出特征是以该垂直线条为中心，向两侧扩展颜色7和8交替的对角线填充模式，形成对称的菱形结构。"
HINTS[347]="这是一个列压缩任务，输入是3x6网格，输出是3x3网格，你需要将每行的前3列与后3列对应位置配对，检查每对中是否有非零值，如果有则输出6，否则输出0。"
HINTS[346]="这是一个路径识别任务，输入和输出的特征是输入图中存在某种颜色的连续路径形成通道或桥梁，你需要识别能够完整穿过网格结构的颜色通道，然后输出该通道对应的颜色值。"
HINTS[344]="这是一个模式替换任务，输入和输出的特征是识别相邻的颜色对(2,3)或(3,2)，将(2,3)替换为(0,8)，将(3,2)替换为(8,0)，其他所有颜色和位置保持不变。"
HINTS[345]="这是一个垂直填充任务，输入和输出的特征是底部有颜色2的标记点，你需要识别输入矩阵底部行的颜色2标记位置，然后从每个标记位置向上填充颜色2的垂直线，直到遇到颜色5的障碍物时停止填充。"
HINTS[342]="这是一个相对位置重映射任务，输入包含一个2x2的8色块和4个其他颜色的像素，你需要识别所有非8颜色的像素，根据它们在整个矩阵中的相对位置关系，将这些像素重新映射到8色块区域的对应相对位置。"
HINTS[343]="这是一个模式扩展任务，输入图的右侧包含一个重复的模式单元，左侧大部分为0，你需要识别这个最小重复单元的模式，然后将其水平重复扩展来填满整个输出网格。"
HINTS[341]="这是一个填充任务，输入和输出的特征是识别包含多种颜色的列，在这些列中找到不同颜色区域之间的空白区域（值为0），然后将这些空白区域填充为颜色8。"
HINTS[339]="这是一个计数和复制的任务，输入和输出的特征是输入为3x3网格，输出为1xN行向量，你需要识别输入中出现次数最多的颜色，然后创建包含N个该颜色的行向量，其中N是该颜色在输入中的出现次数。"
HINTS[340]="这是一个边框吸收任务，输入和输出的特征是具有彩色边框和内部点的网格，你需要识别内部点中颜色与上、下、左或右边框颜色相同的点，将它们移动到对应的边框内部边缘（即第一行/最后一行/第一列/最后一列的相邻内部行/列）并进行颜色值相加，同时将原内部点清除为0。"
HINTS[338]="这是一个轮廓填充任务，输入由颜色0（背景）和颜色2（轮廓）组成，输出需要识别由颜色2形成的闭合矩形轮廓，然后在每个轮廓的内部区域填充颜色3，保持轮廓本身和外部背景为颜色0。"
HINTS[337]="这是一个颜色交换任务，输入和输出的特征是网格中只有颜色5和8相互交换（即5变成8，8变成5），其他所有颜色保持不变，且网格尺寸不变。"
HINTS[336]="这是一个U形填充任务，输入和输出的特征是包含由颜色5构成的U形结构，你需要识别U形的开口方向，然后将U形内部的0区域填充为颜色8。"
HINTS[335]="这是一个路径连接任务，输入和输出的特征是包含颜色2和颜色8的两个特殊点，你需要识别这两个点的位置，然后在它们之间创建一条由颜色4组成的直线路径连接它们。"
HINTS[334]="这是一个模式转换任务，输入是5x5网格包含背景色0和一种其他颜色，输出是3x3网格。你需要识别输入中非零颜色的主要分布区域（左上、中心或右下），然后根据区域位置在输出中生成对应的模式：左上区域对应十字形，中心区域对应加号形，右下区域对应L形，均使用颜色5表示。"
HINTS[333]="这是一个基于参考块的扩展任务，输入包含一个2x2的颜色3参考块和其他分散的颜色点，需要以参考块为基准，将其他颜色点向四周扩展填充，形成连续的色带或填充区域"
HINTS[331]="这是一个模式生成任务，输入和输出的特征是稀疏的1像素点分布在0背景上，你需要识别每个颜色为1的像素位置，然后在其周围生成一个特定的彩色十字图案，使用2,6,7,8等颜色按特定顺序排列，同时保持背景区域的基本结构。"
HINTS[332]="这是一个颜色交替替换任务，输入和输出的特征是3行N列的网格，输入只包含颜色0和5，输出包含颜色0、5和3。你需要识别颜色为5的像素，然后根据列索引的奇偶性执行交替替换操作：偶数列的颜色5保持不变，奇数列的颜色5替换为颜色3。"
HINTS[330]="这是一个连通区域大小分类的任务，输入是由颜色5组成的多个连通区域，输出需要根据每个连通区域的大小进行分类：较小的区域（通常≤3个单元格）标记为颜色2，较大的区域标记为颜色1，背景保持颜色0。"
HINTS[329]="这是一个网格处理任务，输入和输出的尺寸相同，你需要识别网格的中间列，保留中间列的值，并将其他所有位置的值设置为0。"
HINTS[327]="这是一个对角线传播任务，输入是3x3矩阵，输出是6x6矩阵，你需要识别输入矩阵中的非零元素序列，然后将这个序列沿着输出矩阵的对角线方向重复排列，其余位置填充0。"
HINTS[328]="这是一个基于种子点的区域划分任务，输入网格中有少量不同颜色的种子点，输出需要根据曼哈顿距离最近原则将网格划分为对应颜色的区域，当距离相近时根据特定规则选择颜色。"
HINTS[326]="这是一个裁剪任务，输入和输出的特征是输入为各种尺寸的矩阵而输出固定为2x2矩阵，你需要识别输入矩阵的左上角2x2区域，然后提取该区域作为输出。"
HINTS[325]="这是一个连通区域计数和表示的任务，输入图中包含多个颜色为8的连通区域（背景为0），你需要识别所有连通区域的数量n，然后输出一个n×n的对角矩阵，对角线上的值为8，其余位置为0。"
HINTS[324]="这是一个边界对角线检测和颜色替换的任务，输入和输出的特征是相同尺寸的网格，你需要识别位于网格边界和对角线上的特定单元格，然后根据邻居的颜色分布将这些单元格的颜色替换为新的颜色值。"
HINTS[323]='这是一个路径扩展任务，输入图中有一个颜色8的点，你需要从该点开始按照"向下移动两步，向左移动两步"的循环模式向外扩展，将路径上的点染成颜色5，同时保留原始的颜色8点。'
HINTS[322]="这是一个像素传播任务，输入和输出的特征是3x3网格中包含稀疏的非零像素和大量零值背景，你需要识别非零像素的位置，然后从上向下传播非零颜色值（每个位置取自身和上方相邻位置的最大值），实现颜色值的垂直扩散填充。"
HINTS[321]="这是一个列提取任务，输入是4x14网格，输出是4x4网格，需要从每行的第0、4、9、13列位置提取对应的值来构建输出网格。"
HINTS[320]="这是一个列操作任务，输入和输出的特征是每列中有从底部向上延伸的颜色2像素，你需要计算每列中颜色2像素的数量除以4，然后将该列从底部开始相应数量的颜色2像素转换为颜色8。"
HINTS[319]="这是一个图案提取任务，输入是一个大网格包含背景色和多个几何图案，你需要识别出非背景色形成的特定几何形状（如十字形、H形等），然后提取该图案并缩放到较小的输出网格中，保持图案的形状和颜色特征。"
HINTS[318]="这是一个二进制逻辑操作任务，输入是一个9x4的网格，其中第5行是分隔行，你需要将上半部分（前4行）和下半部分（后4行）进行逐元素OR操作，然后将结果为True的位置输出为颜色3，False的位置输出为颜色0，生成4x4的输出网格。"
HINTS[316]="这是一个网格提取和重排任务，输入是10x10网格，其中散布着少量非零颜色点，输出是3x3网格。你需要从输入网格的每一列中提取最大值（忽略0值），然后将这些值按特定顺序排列成3x3的输出网格，不足9个时用0填充。"
HINTS[315]="这是一个矩阵复制和位置映射的任务，输入是3x3矩阵，输出是9x9矩阵，需要根据输入矩阵中非零元素的分布模式，将输入矩阵复制并放置到输出网格的特定位置（通常是角落区域），其余区域填充为0。"
HINTS[317]="这是一个区域检测与合并的任务，输入是稀疏的5在0背景上，每个5点表示一个3x3区域（左上角在该点位置减1），输出需要计算所有此类区域的并集（合并相邻或重叠区域）并填充为1，其余为0"
HINTS[313]="这是一个边框移除和模式扩展的任务，输入的特征是包含一个边框区域和内部棋盘格模式，你需要识别并移除边框区域，然后将内部的棋盘格交替模式扩展到整个输出矩阵。"
HINTS[314]="这是一个模式传播任务，输入和输出的特征是网格中有特殊颜色值（>1）的像素点，你需要识别这些特殊颜色值，当两个相同特殊颜色值间隔6格（水平或垂直方向）时，在它们中间位置（间隔3格）复制该特殊颜色值。"
HINTS[312]="这是一个行首颜色传播的任务，输入包含颜色5的块和其他颜色，你需要将每行中所有非零单元格替换为该行最左侧非零单元格的颜色值，同时保持背景色0不变"
HINTS[311]="这是一个网格扩展任务，输入是3x3网格，输出是3x6网格，你需要将每一行水平镜像（反转）后拼接在原行的右侧。"
HINTS[310]="这是一个尺寸缩减和边框提取任务，输入是大尺寸矩阵包含重复模式，输出是小尺寸矩阵。你需要识别出现次数最少的颜色来确定边框颜色和内部尺寸，然后构建一个边框为该颜色、内部填充来自输入矩阵对应位置的数值的输出矩阵。"
HINTS[309]="这是一个颜色替换任务，输入和输出的特征是尺寸相同且颜色7的位置被替换为颜色5，其他颜色保持不变。你需要识别所有颜色为7的像素，然后将它们替换为颜色5"
HINTS[308]="这是一个对称提取任务，输入的特征是有一个主要背景颜色和多个稀疏标记颜色对称分布，你需要识别这些标记颜色的位置并计算它们的几何中心，然后构建一个以中心为原点的对称网格，将标记颜色映射到对应的对称位置上。"
HINTS[307]="这是一个尺寸放大的任务，输入和输出的特征是输入为n×n矩阵，输出为2n×2n矩阵，你需要将每个输入像素复制到对应的2×2输出块中，实现矩阵的双倍放大。"
HINTS[305]="这是一个模式填充任务，输入和输出的特征是在16x16网格中包含0值区域和数字序列，你需要识别第一行数字序列的最大值来确定循环模式，然后用(i+j)%max_value+1的公式填充所有0值区域。"
HINTS[304]="这是一个网格复制和模式选择的任务，输入是3x3网格，输出是9x9网格，你需要识别输入矩阵中出现最频繁的颜色，然后在输出网格的3x3分区中，只有当对应输入位置的颜色等于最频繁颜色时，才在该分区复制输入矩阵的内容，否则填充0。"
HINTS[306]="这是一个对称填充的任务，输入有一个全为4的中间行作为对称轴，你需要将上半部分的非零颜色点以该行为轴垂直镜像到下半部分。"
HINTS[302]="这是一个矩形框内部填充的任务，输入图中包含由颜色5组成的矩形框，你需要根据矩形框在网格中的位置（左上/右下）来填充内部区域，右下位置的矩形填充颜色8，左上位置的矩形填充颜色7，并在最大的矩形中心位置填充颜色6"
HINTS[303]="这是一个标记任务，输入和输出的尺寸相同，你需要识别输入中所有值均为0的行和列，将这些行和列的所有单元格设置为颜色2，其他单元格保持不变。"
HINTS[300]="这是一个对象提取任务，输入和输出的特征是输入包含多个颜色对象但有一个主要非背景色对象，你需要识别出现频率最高的非0颜色，然后提取包含该颜色的所有行和列的交集区域作为输出。"
HINTS[301]="这是一个重力沉降任务，输入网格中有一些非零颜色块（物体）分布在不同的行，你需要识别所有非零行（物体），然后将这些非零行按原始顺序移动到网格底部（即输出中非零行在底部且顺序与输入中从上到下出现顺序相同），全零行则留在顶部。"
HINTS[298]="这是一个颜色循环置换任务，输入和输出的特征是具有三层嵌套矩形结构，你需要识别最外层、中间层和最内层的颜色，然后将最外层颜色变为最内层颜色，中间层颜色变为最外层颜色，最内层颜色变为中间层颜色，保持原有的嵌套结构不变。"
HINTS[299]="这是一个网格扩展任务，输入中有一些8（在同一列）和2（在同一行），你需要识别包含2的行和包含8的列，将它们的交点变为4，并将2所在的行全部变为2（除交点外），8所在的列全部变为8（除交点外）。"
HINTS[297]="这是一个模式复制的任务，输入的特征是前两行包含非零颜色值（第一行多个颜色，第二行全为5），其余行为0；输出的特征是需要保持前两行不变，然后按照第一行的颜色顺序，将每个颜色扩展为整行填充，并重复这个颜色序列直到填满剩余行。"
HINTS[296]="这是一个特征提取任务，输入是5x7网格，输出是3x3网格，你需要从输入的首行、中间行（第1和3行的列最大值）和末行中提取第0、1、5、6列，并对第1列和第5列进行按位或运算来生成输出。"
HINTS[295]="这是一个矩阵扩展任务，输入是单行向量包含非零值和零值，输出需要生成多行矩阵，行数等于输入中非零值的数量，每行向右逐步增加非零值的覆盖范围，其余位置保持为零"
HINTS[294]="这是一个区域填充任务，输入和输出的特征是由颜色5组成的封闭区域，你需要识别这些区域的内部位置（非边界），然后将这些内部位置的颜色从5改为2。"
HINTS[293]="这是一个网格转换任务，输入和输出的特征是包含垂直分隔线和水平区域，你需要识别垂直分隔线列和与其相交的水平区域行，然后将水平区域的颜色扩展到垂直分隔线与水平区域相交的单元格中。"
HINTS[291]="这是一个识别特定形状的任务，输入是包含多个颜色矩形区域的网格，输出是1x1的单值。你需要识别内部包含空洞（有0像素）的矩形区域，然后输出该区域的颜色值。"
HINTS[292]="这是一个数值转换任务，输入和输出的特征是3行n列的网格，包含颜色0和4，你需要识别每3列的第一个位置（列索引为0,3,6,...），然后将这些位置的数值乘以1.5（4变成6，0保持0）。"
HINTS[290]="这是一个矩形框颜色交换并缩小的任务，输入中包含一个由两种颜色组成的矩形框结构（边框和内部），你需要识别这个矩形框，交换边框和内部的颜色，然后输出一个缩小版本的矩形框。"
HINTS[289]="这是一个网格放大任务，输入是3x3网格，输出尺寸由输入中不同非零颜色的数量决定（n种颜色则放大n倍），你需要将每个输入单元格根据其颜色值放大为n×n的块，0颜色保持为背景。"
HINTS[288]="这是一个对称填充任务，输入图的底部区域包含一个非零颜色的图案，你需要识别底部图案的中心颜色，然后在顶部区域对称地填充该颜色，形成关于垂直轴对称的完整图案。"
HINTS[287]="这是一个颜色替换任务，输入和输出的特征是16x16的对称网格，你需要识别颜色为4的像素，然后根据对称位置或其他规则将其替换为适当的颜色值。"
HINTS[286]="这是一个区域填充任务，输入图中有由8构成的边界和0构成的背景区域，你需要识别特定的种子点（非0非7的像素），然后按照棋盘格模式（基于位置奇偶性交替使用两种颜色）填充所有与种子点连通的0区域。"
HINTS[285]="这是一个对称扩展任务，输入包含稀疏的颜色种子像素，输出需要在保持网格尺寸不变的情况下，识别这些种子像素的颜色，然后基于对称中心点向四个对称方向（水平、垂直、对角线）扩展填充相同的颜色，形成对称的彩色图案。"
HINTS[284]="这是一个沙漏形状生成任务，输入包含两个不同颜色的点，你需要识别这两个点的位置，然后在这两点之间生成一个对称的沙漏形状，其中一个颜色向上/左扩展，另一个颜色向下/右扩展，在中间区域形成对称的填充模式。"
HINTS[283]="这是一个边框转换任务，输入和输出的特征是输入中只有0和5两种颜色，输出中0保持不变，5色块被转换为特定的边框图案：四个角为1，边缘中间为4，中心为2，需要识别5色块的邻域关系，然后执行相应的边框绘制操作。"
HINTS[282]="这是一个模式扩展任务，输入是稀疏分布的颜色5点阵，输出需要在每个5色点周围创建一个3x3的十字模式：中心保持0，上下左右四个方向设为5，四个对角线方向设为1，多个点的扩展模式可以重叠。"
HINTS[281]="这是一个矩形扩展任务，输入图中包含一个双层矩形结构（外框和内框颜色不同）和一个标记点8，你需要根据标记点8的位置确定扩展方向，将内框区域向外扩展，保持外框颜色不变但扩大外框范围"
HINTS[280]="这是一个区域扩展任务，输入图中包含颜色3的区域和内部的颜色2种子点，你需要识别颜色2点的位置，确定其所在颜色3区域的边界，然后沿着较长轴方向用颜色2扩展填充该区域，其他位置用颜色3填充。"
HINTS[278]="这是一个邻域扩展任务，输入包含背景(0)和对象点(2)，需要识别所有颜色为2的像素，然后在其8邻域（包括对角线方向）的所有有效位置（不覆盖原有的2）填充颜色3。"
HINTS[277]="这是一个连通分量分类任务，输入由背景色0和前景色8组成，你需要识别所有颜色8的连通区域，根据连通区域的形状特征（如大小、复杂度等）将其分为两类，分别用颜色1和颜色2标记。"
HINTS[279]="这是一个连通区域标记和重新染色的任务，输入图中颜色9是背景，颜色1是目标区域。你需要识别所有与背景（颜色9）相邻的颜色1的连通区域，并将这些连通区域的颜色改为8（而完全被1包围、不与背景相邻的区域保持为1）。"
HINTS[276]="这是一个颜色替换任务，输入和输出的特征是尺寸相同且颜色7保持不变，你需要识别所有颜色为6的像素，然后执行将所有6替换为2的操作。"
HINTS[275]="这是一个图案生成任务，输入图包含颜色8（标记）和其他颜色（1-6），你需要识别每个颜色8与其他颜色的配对关系，然后在输出放大网格的对应块内绘制一个十字形（中心点加四个方向延伸），颜色为配对的非8颜色。"
HINTS[274]="这是一个模式映射任务，输入和输出的特征是一个由颜色5构成的边框包围着颜色8的内部区域，你需要识别内部颜色8区域的大小，然后根据其尺寸在3x3输出网格中从左上角开始填充相应数量的8，其余位置填充0。"
HINTS[273]="这是一个连接标记点的任务，输入和输出的特征是输入中有颜色4的标记点，输出中增加了颜色2的连接区域，你需要识别同一行中成对出现的4标记点，然后在它们之间的垂直区域填充颜色2来连接这些标记点。"
HINTS[272]="这是一个识别孤立单元格的任务，输入和输出的特征是网格尺寸相同，主要包含颜色0和2，你需要识别那些颜色为2且周围没有其他非0单元格（即孤立）的位置，将这些孤立的位置颜色从2改为1。"
HINTS[270]="这是一个点移动任务，输入和输出的特征是包含颜色1、2、3、7的稀疏点阵，你需要识别颜色1和2作为基准点，然后将颜色3和7的点分别向对应的基准点移动，形成以基准点为中心的十字形图案。"
HINTS[271]="这是一个模式提取任务，输入是9x9网格包含颜色0、8、1，输出是3x3网格只包含颜色8和1，你需要识别输入中具有特定统计特征（如最小颜色和）的3x3子区域，然后提取该区域作为输出。"
HINTS[269]="这是一个网格放大任务，输入是3x3网格，输出是放大后的网格，放大倍数等于输入中非零颜色的数量。每个非零颜色在输出中形成一个n×n的方块（n=放大倍数），这些方块按照输入中的位置顺序排列在输出网格的对应区域中。"
HINTS[268]="这是一个形状填充与喷泉外扩任务，输入和输出图像的特征是背景为0、轮廓为非零颜色、内部及喷泉均为颜色4，你需要识别半开口轮廓，执行内部填充（颜色4），并在开口处发出垂直喷流、从边缘位置发出对角喷流。"
HINTS[267]="这是一个颜色替换任务，输入和输出的特征是左下角有一个特殊颜色标记，你需要识别输入中所有非零像素（通常是某种特定形状），然后将这些非零像素的颜色替换为左下角标记的颜色，最后将左下角的标记位置清零。"
HINTS[266]="这是一个基于标记点生成对称图案的任务，输入图中有一个颜色为2的标记点，你需要以该标记点为中心，在特定相对位置生成对称的颜色3、6、7、8的图案。"
HINTS[265]="这是一个模式检测和标记任务，输入是包含颜色5和0的网格，你需要识别完全由0组成的2×2方块区域（且该区域至少有一边与颜色5相邻），然后将这些2×2区域的所有0填充为颜色2。"
HINTS[264]="这是一个模式提取和重组的任务，输入网格包含多个被颜色5包围的3x3彩色块，你需要识别这些特定的3x3模式（通过二进制编码特征值匹配），然后将它们按特定顺序重新排列成一个3x3的输出网格，每个位置放置一个对应的3x3彩色块。"
HINTS[263]="这是一个模式提取任务，输入是包含多个颜色模式的大尺寸网格，输出是3x3网格。你需要识别输入中出现频率最低的颜色所对应的模式区域，然后提取包含该颜色的一个3x3区块作为输出。"
HINTS[262]="这是一个位置映射任务，输入是包含颜色5的3x3网格，输出需要将每一行转换为单一颜色，该颜色由该行中颜色5所在的列位置决定（如果没有颜色5，则基于特定规则），映射关系为列位置对应输出颜色（如列0→2，列1→4，列2→3）。"
HINTS[261]="这是一个移动并变色的任务，输入图中的所有颜色8的像素需要向下移动一行（如果下方有边界则停止），并将这些像素的颜色改为2，其他像素保持不变。"
HINTS[260]="这是一个对角线扩展任务，输入的特征是包含一条主对角线和附近的辅助区域，你需要识别主对角线的颜色和位置范围，然后在主对角线两侧创建平行的对角线（通常偏移±2个单位）。"
HINTS[259]="这是一个目标区域提取任务，输入和输出的特征是输入包含背景色1和多个非1的目标区域，你需要识别所有非1的连通区域，将这些区域提取到输出中，保持它们的相对位置关系，并将原背景色1转换为0作为新背景。"
HINTS[258]="这是一个路径连接任务，输入和输出的特征是识别水平方向上相邻的颜色1（特征点），并在它们之间的空白位置（颜色0）填充颜色2作为连接线。"
HINTS[257]="这是一个区域提取与合并的任务，输入被颜色1的水平线分为上下两部分，你需要从上半部分的前4列和下半部分的前4列提取对应位置的值，通过逻辑OR操作合并生成4×4的输出网格"
HINTS[256]="这是一个对角线填充任务，输入中有颜色为2的障碍物块，你需要从左上角开始向右下角进行对角线填充，使用颜色3填充靠近对角线的区域，颜色1填充远离对角线的区域，并在遇到2色障碍物时停止填充。"
HINTS[255]="这是一个矩形填充任务，输入和输出的特征是网格中已有特定颜色的图案，你需要识别网格中最大的连续全零矩形区域，然后用颜色3填充该区域，但填充不能覆盖已有的非零元素。"
HINTS[254]="这是一个列标记任务，输入和输出的特征是识别由颜色5组成的连续垂直列，将该列标记为颜色1，并在底部区域根据特定条件将某些位置标记为颜色2。"
HINTS[253]="这是一个形状提取和重排任务，输入是13×13网格中分布在四个角落的L形形状（每组3个同色像素），输出需要将这些L形提取并重新排列到4×4网格的四个角落，保持每个形状的颜色和结构不变，中心区域填充0。"
HINTS[252]="这是一个对角线模式识别任务，输入和输出的特征是包含一条或多条对角线上的前景色点，你需要识别这些对角线上的点，然后将对角线上的每隔一个前景色点替换为颜色4。"
HINTS[251]="这是一个区域填充任务，输入和输出的特征是包含由颜色2形成的边界结构，你需要识别所有被颜色2完全包围的闭合区域，然后将这些区域内部的颜色0像素填充为颜色1。"
HINTS[250]="这是一个位置重排任务，输入中有2x2的矩形块（颜色2）和多个散点（颜色5），你需要以2x2矩形块为中心，将所有颜色5的点重新分布到该矩形块周围的3x3邻域范围内"
HINTS[249]="这是一个水平镜像复制的任务，输入和输出的特征是输出网格的宽度是输入的两倍而高度不变，你需要将输入的每一行水平重复一次来生成输出。"
HINTS[248]="这是一个模式生成任务，输入图中只有一个颜色为1的像素点作为起始位置，你需要生成一个对称的反射模式，其中颜色为1的像素点从起始位置开始沿着对角线方向移动并在边界反射，形成连续的弹跳轨迹。"
HINTS[246]="这是一个路径连接任务，输入网格中有两个特殊点（颜色2和颜色3），你需要在这两个点之间绘制一条L形的最短路径（先水平后垂直或先垂直后水平），将路径上的所有点填充为颜色8。"
HINTS[247]="这是一个颜色统计和重组任务，输入是稀疏颜色分布的网格，输出需要提取所有非零颜色并按出现频率排序，然后创建行数为最小颜色出现次数的网格，每行包含所有提取的颜色。"
HINTS[245]="这是一个图形对称变换任务，输入和输出的特征是包含固定标记点（颜色3）和可移动图形（颜色2），你需要识别所有颜色为2的像素点，然后以第一个非2非3的点为参考，将所有2像素点进行对称平移操作。"
HINTS[243]="这是一个洪水填充任务，输入和输出的特征是尺寸相同，你需要识别所有与颜色1相邻的0像素，然后将这些0像素填充为颜色1，实现从现有1像素开始的区域增长。"
HINTS[244]="这是一个模式提取任务，输入是由交叉线条分割成多个区域的网格，你需要识别网格中的重复单元和采样步长，然后提取代表性模式形成小尺寸的输出网格。"
HINTS[241]="这是一个矩阵转置任务，输入和输出的尺寸相同，主对角线上的颜色5保持不变，其他元素的位置按照矩阵转置的规则进行行列互换。"
HINTS[242]="这是一个位置提取任务，输入和输出的特征是输入网格中包含0值区域，你需要识别0值区域的位置，然后提取该区域左上角相邻的3x3区域作为输出。"
HINTS[240]="这是一个对称扩展任务，输入是稀疏的点阵，输出需要基于输入点创建关于网格中心对称的完整模式，将输入点作为对称轴或对称中心的参考点，填充相应的对称位置。"
HINTS[239]="这是一个频率可视化任务，输入是一个包含多种颜色的矩阵，输出需要按颜色频率降序排列，构建一个阶梯状的直方图，高频颜色出现在更多行，低频颜色出现在较少行，用0填充空缺位置。"
HINTS[238]="这是一个几何变换任务，输入包含多个矩形框和一个颜色8的十字结构，需要以颜色8的位置为参考点，将输入中的矩形框重新排列成一个对称的菱形输出结构，并保持颜色对应关系。"
HINTS[237]="这是一个区域填充任务，输入和输出的特征是在0背景上有多个不同颜色的种子点，你需要识别每个非零颜色的种子点位置，然后执行向右和向下方向填充相同颜色的操作，将相邻的0值单元格替换为种子点的颜色。"
HINTS[235]="这是一个模式识别任务，输入是4x14的网格被垂直分隔符分成三个4x4区域，每个区域由颜色5和0组成，你需要检测每个区域中第1行中心列和第2行左侧/中心列是否为0，根据这些条件的组合输出对应的颜色（2,3,4,8）到3x3的输出网格中。"
HINTS[236]="这是一个网格转换任务，输入是9x4网格，其中第5行是分隔行[4,4,4,4]，你需要将前4行和后4行对应位置的颜色值进行配对，根据它们的和是否为3的倍数来决定输出颜色（0或3），最终生成4x4的输出网格。"
HINTS[234]="这是一个图形重组任务，输入包含多个彩色区域，你需要识别主要的矩形区域和需要移动的次要结构，然后将次要结构移动到与主要结构对齐的位置，并移除原始位置的元素。"
HINTS[232]="这是一个条纹延伸任务，输入和输出的特征是稀疏的非零像素分布在背景0中，你需要识别每个非零像素的位置，然后从该像素开始向右水平延伸交替颜色的条纹（原始颜色和颜色5交替），直到网格边界。"
HINTS[233]="这是一个模式提取和重组任务，输入包含一个主结构和多个嵌入的3x3模式块，你需要识别所有包含颜色2和其他非零颜色的3x3模式块，将它们从原位置移除，然后通过旋转匹配将这些模式块重新插入到裁剪后的主结构网格中。"
HINTS[231]="这是一个水平扩展的任务，输入和输出的特征是行数保持不变而列数翻倍，你需要识别每行中的非零内容模式，然后将该模式水平重复一次以生成输出。"
HINTS[230]="这是一个标记任务，输入和输出的特征是包含一个或多个2×2的颜色5的矩形块，你需要识别这些2×2色块，然后在每个色块的左上对角方向添加颜色1和2，在右下对角方向添加颜色3和4。"
HINTS[229]="这是一个颜色替换任务，输入和输出的尺寸相同，你需要识别输入网格中出现频率最高的颜色，保持该颜色不变，将其他所有颜色都替换为颜色5。"
HINTS[228]="这是一个对称映射任务，输入和输出的特征是一个包含矩形框和内部特殊点的网格，你需要识别矩形框的边界，然后将框内的特殊颜色点对称映射到框外对应位置，并将原位置清零。"
HINTS[226]="这是一个区域填充任务，输入是由颜色5的线条分隔的不同区域，你需要识别这些被分隔的区域，然后将左上角区域填充为1，中间区域填充为2，右下角区域填充为3。"
HINTS[227]="这是一个网格压缩任务，输入是8x4网格，输出是4x4网格，需要将输入的前4行和后4行对应位置相加，如果和为0则输出2，否则输出0。"
HINTS[225]="这是一个镜像扩展任务，输入图的中心有一个2x2的非零颜色块，你需要将这个2x2块的颜色按对角方向镜像复制到四个角落区域，生成四个新的2x2颜色块，保持中心块不变。"
HINTS[224]="这是一个矩形扩展任务，输入包含一个特定颜色的内部矩形和几个颜色为5的外部参考点，你需要识别内部矩形的边界和颜色，然后在其周围创建一个更大的同颜色矩形边框，同时保持所有颜色为5的点位置不变。"
HINTS[222]="这是一个区域提取任务，输入和输出的特征是相同尺寸的网格，你需要识别输入中最大的连续同色矩形区域，然后在输出中只保留该区域的原色，其他所有位置都设置为0。"
HINTS[221]="这是一个网格扩展任务，输入是3x3网格，输出是更大的网格，需要根据输入中非零元素的数量确定输出尺寸，将输入网格以某种模式平铺在输出网格的左上角区域，其余区域用0填充。"
HINTS[220]="这是一个基于种子点的区域填充任务，输入的特征是稀疏的非零点分布在背景0中，输出的特征是在每个非零点周围创建3×3的填充方块，中心保持原色，周围使用特定映射颜色填充（8→4，3→6，2→1）。"
HINTS[223]="这是一个放大的任务，输入是3x3网格，输出是9x9网格，你需要将每个输入像素替换为一个3x3的块，块内所有像素颜色与原输入像素相同，然后按原顺序排列成输出。"
HINTS[219]="这是一个模式延伸的任务，输入包含颜色8形成的各种几何形状，你需要识别这些形状的模式特征，然后在特定位置（通常是右侧或下方）以相同的几何模式延伸颜色1的像素序列。"
HINTS[218]="这是一个区域提取任务，输入是一个大网格包含多个矩形色块区域（背景为0），输出是提取包含非零内容的最小边界矩形，并保留该区域内各位置的原色值。"
HINTS[217]="这是一个模式扩展任务，输入中的非零点定义了一个3x3网格内的模板模式，输出需要将这个模板模式复制到所有3x3网格块中，保持每个点在其所属3x3块内的相对位置不变。"
HINTS[216]="这是一个区域提取任务，输入是一个20×20的网格，包含背景色0、主色1和特殊色2，你需要识别包含最多特殊色2的连续矩形区域，然后提取该区域作为输出。"
HINTS[215]="这是一个模式重复任务，输入的特征是包含一个由三行组成的特定模式（稀疏值行、全值行、稀疏值行），你需要识别这个三行模式的起始位置，然后将这个模式垂直重复填充到整个输出矩阵中。"
HINTS[213]="这是一个条带提取和重构任务，输入包含多个水平或垂直的有色条带，你需要识别主要条带的方向和颜色，然后构建一个正方形输出网格，其中每行（水平条带）或每列（垂直条带）填充对应条带的颜色。"
HINTS[214]="这是一个矩阵对称填充任务，输入矩阵左侧有一个3x3的彩色区域，中间和右侧有大量0值，你需要识别颜色5作为对称轴标记，然后将左侧3x3区域的内容以中心对称的方式复制填充到右侧的0值区域。"
HINTS[211]="这是一个对称扩展任务，输入是3×2矩阵，输出是9×4矩阵，需要将输入矩阵进行垂直翻转+原矩阵+垂直翻转的三重堆叠，然后对每一行进行水平翻转+原行的双重扩展，生成对称的输出模式"
HINTS[210]="这是一个矩阵变换任务，输入是3x3的网格，输出是6x3的网格，你需要将输入矩阵与其垂直翻转（上下翻转）的版本进行垂直拼接。"
HINTS[209]="这是一个内容提取和重组任务，输入图的四个角落有颜色4标记边界，你需要识别边界内的所有非零非4的块状内容，然后通过寻找合适的缩放因子和偏移量，将这些内容重新映射到输出网格中，同时保持输出图的四个角落仍然是颜色4。"
HINTS[212]="这是一个线段延伸任务，输入中有一条颜色5的水平线（一整行），以及一些颜色1或2的点。你需要从每个点出发，沿着列向水平线的方向（上半部分的点向下，下半部分的点向上）延伸，用点的颜色填充路径上的0，直到遇到水平线或边界。"
HINTS[208]="这是一个边框标记任务，输入和输出的特征是尺寸相同但某些区域被添加了边框，你需要识别出现次数最少的非零颜色，找到该颜色定义的矩形区域尺寸，然后在图像中寻找相同尺寸的全零区域，并用该颜色为其添加边框。"
HINTS[207]="这是一个模式提取任务，输入是5x5网格包含背景色0和另一种颜色，输出是2x2网格，你需要识别输入中最频繁出现的非全零2x2子网格模式，然后输出该模式。"
HINTS[206]="这是一个图案复制的任务，输入和输出的特征是包含一个颜色为5的标记点和一组由颜色1,2,3,6等组成的图案块，你需要识别图案块的位置和颜色5标记点的位置，然后将图案块复制到标记点附近的位置"
HINTS[205]="这是一个网格提取和模式扩散任务，输入是包含多种颜色的大网格，输出是只包含两种颜色的小网格。你需要识别包含最多两种颜色的最大矩形区域，裁剪该区域，然后将次要颜色在行和列方向上进行扩散，形成网格状的模式分布"
HINTS[204]="这是一个矩形内部填充任务，输入是由颜色1形成的矩形轮廓和颜色0的背景，你需要识别所有闭合的矩形区域，根据矩形的大小关系，将较大的矩形内部填充为颜色2，较小的矩形内部填充为颜色7，同时保持原始轮廓不变。"
HINTS[203]="这是一个颜色映射反转的任务，输入和输出的特征是多层同心矩形结构，你需要识别从外到内的颜色序列，然后将这个颜色序列反转，保持相同的空间结构但颜色顺序相反。"
HINTS[202]="这是一个区域扩展任务，输入由多个颜色区域组成，其中包含0值像素，你需要识别0值像素所在的行或列，并将该行/列中与0值像素相同颜色的所有像素都变为0，扩展方向取决于0值像素在水平还是垂直带状区域中。"
HINTS[201]="这是一个边界裁剪和内容提取的任务，输入图中包含由颜色4标记的边界框架，你需要识别这个边界内的有效内容（非零像素），然后提取这些内容并重新排列成一个更紧凑的矩形，同时保持颜色4作为输出边界。"
HINTS[199]="这是一个模式生成任务，输入图中有一个特殊的非零颜色点，你需要在该点所在行及以上的所有行创建棋盘格模式（4和0交替），保持特殊点的原值不变，特殊点以下的行保持不变。"
HINTS[200]="这是一个网格传播任务，输入和输出的特征是输入矩阵最后一行有一个非零颜色点，你需要识别该点的颜色和水平位置，然后从该位置开始每隔一列创建垂直的颜色列，并在第一行和最后一行的特定交叉位置添加颜色5。"
HINTS[197]="这是一个模式扩展任务，输入和输出的特征是某些行包含前两个数字组成的模板模式，你需要识别这些模板行，然后将同一行中的零值位置按照前两个数字的交替模式进行填充扩展。"
HINTS[198]="这是一个网格分区着色任务，输入中有特定颜色形成的十字交叉线条将网格划分为多个区域，你需要识别这些线条形成的网格结构，然后以棋盘状交替模式用颜色3和4填充不同的区域，同时保持原始线条颜色不变。"
HINTS[196]="这是一个形状识别的任务，输入和输出的特征是识别由颜色1组成的完整矩形区域（包括空心和实心矩形），将这些矩形区域的边框颜色从1改为3，同时保持矩形内部和外部区域不变。"
HINTS[195]="这是一个空间映射任务，输入是包含多个颜色为5的矩形块的网格，输出是固定的9x9网格。你需要识别输入中颜色为5的矩形块的分布位置，将它们映射到3x3的网格划分中，然后根据每个子网格中5的分布特征，在输出的9x9网格中生成相应的交叉模式。"
HINTS[193]="这是一个形态学过滤任务，输入和输出的特征是相同的网格尺寸，你需要识别孤立的像素点（颜色值不为0），然后将那些周围相同颜色邻居数量不足的孤立像素移除（设为0），只保留密集的色块区域。"
HINTS[192]="这是一个噪声移除任务，输入图中存在少量噪声颜色点，你需要识别这些噪声点（出现频率最低的颜色），然后检查每个噪声点是否与主要颜色区域（出现频率次低的颜色）相邻，如果相邻则保留，否则将其替换为主要颜色或背景色。"
HINTS[191]="这是一个模式识别与扩展任务，输入图中包含由颜色1和4组成的特定十字形模式，你需要识别这些模式的位置和方向，然后在其他位置寻找相同的模式匹配，并在匹配模式的周围8邻域区域将背景色0填充为颜色1。"
HINTS[194]="这是一个网格放大任务，输入是3x3网格，输出是6x6网格，你需要通过将输入网格及其反射（垂直和水平）副本组合成2x2布局来生成输出。"
HINTS[190]="这是一个反射延伸任务，输入和输出的特征是包含不完整反射图案的网格，你需要识别反射对称中心点和反射方向，然后沿着反射方向延伸颜色以完成完整的反射图案。"
HINTS[189]="这是一个网格分割重组任务，输入是一个9×9网格，其中有一条由颜色8组成的水平或垂直分隔带将网格分割成不同区域，你需要识别分隔带的位置，然后从被分割的不同区域中提取3×3的模式块，并将这些块重新组合成一个6×6的输出网格。"
HINTS[187]="这是一个区域填充任务，输入的特征是有特定颜色路径形成的迷宫状结构，输出的特征是将路径外部背景填充为颜色3，路径内部的封闭区域填充为颜色2，同时保持路径本身的颜色不变。"
HINTS[188]="这是一个模式提取任务，输入是输出的重复模式（水平或垂直方向），你需要检测重复方向，然后提取原始的非重复模式部分"
HINTS[186]="这是一个计数和位置映射任务，输入和输出的特征是3x3网格且尺寸相同，你需要统计输入中颜色1的数量，然后在输出的第一行从左到右放置相应数量的颜色2（最多3个），其余位置填充0，并在第二行中间位置根据数量计算一个特殊值。"
HINTS[185]="这是一个模式提取任务，输入是大尺寸的网格（27x27或29x29），输出是3x3网格，需要识别输入网格中特殊颜色单元格的分布模式，提取这些特殊颜色值并按它们在网格中的相对位置关系构建3x3输出矩阵。"
HINTS[184]="这是一个区域提取任务，输入图被全0行分隔成多个区域，每个区域包含不同颜色的连通块，你需要识别每个区域中占据主导地位的颜色（最大连通块的颜色），然后按区域顺序排列成输出矩阵。"
HINTS[182]="这是一个图案颜色匹配任务，输入和输出的特征是有颜色5组成的矩形框标记内部图案，你需要识别框内图案的形状和颜色，然后在框外找到相同形状的图案，将其颜色改为框内图案的颜色。"
HINTS[181]="这是一个图案扩展任务，输入和输出的特征是包含颜色8的可扩展区域和颜色4的固定参考区域，你需要识别颜色8的现有区域，然后根据颜色4参考区域的位置关系，将颜色8向远离参考区域的方向扩展填充。"
HINTS[183]="这是一个基于象限的角点颜色传播任务，输入是一个带有边框和内部图案的网格，四个角有特殊颜色。你需要提取中心区域（去除边框后），对于中心区域中每个非零单元格，根据其所在象限（将中心区域分为四等分）用对应的角点颜色（左上、右上、左下、右下）替换，零值保持不变，最后输出一个尺寸减半的网格。"
HINTS[179]="这是一个矩阵变换任务，输入和输出的特征是具有相同颜色分布的3x3网格，你需要识别输入矩阵的排列方式，然后执行矩阵转置操作来得到输出矩阵。"
HINTS[180]="这是一个下采样任务，输入是8x8网格分为上下两个4x8区域，输出是4x4网格，需要对每个2x2块（跨越上下区域）按右上→左下→右下→左上的顺时针顺序选择非零值进行下采样。"
HINTS[177]="这是一个边界框提取任务，输入是一个包含大量0背景和若干非零颜色组成的形状的矩阵，输出需要提取包含所有非零颜色的最小边界矩形，去除所有全零的行和列，保持原始非零颜色的相对位置不变。"
HINTS[178]="这是一个矩阵压缩任务，输入是包含重复行或渐变行的矩阵，你需要识别行模式的变化：当各行不同时提取每行的第一个元素形成列向量，当所有行相同时压缩为单行并去除相邻重复值。"
HINTS[175]="这是一个基于位置的颜色替换任务，输入和输出的特征是相同尺寸的网格，其中包含颜色0作为特殊标记，你需要识别所有颜色0的位置，然后根据行列位置计算并替换为特定的非零颜色值，这些值沿着主对角线方向呈现规律性变化。"
HINTS[176]="这是一个模式识别的任务，输入和输出的特征是3行网格，你需要识别每个位置是否满足特定的空间模式条件，然后将符合条件的0值替换为颜色4。"
HINTS[174]="这是一个形状提取任务，输入包含多个颜色块，你需要识别其中具有对称性（水平、垂直或旋转对称）的特定颜色块，然后提取该颜色块的边界框区域作为输出。"
HINTS[172]="这是一个矩阵镜像扩展任务，输入是3×3的矩阵，输出需要将输入矩阵与其垂直翻转版本进行拼接，形成6×3的输出矩阵。"
HINTS[173]="这是一个对称扩展的任务，输入图中包含一些孤立的非零像素点或小的对称模式，你需要识别这些对称模式的中心或端点，然后沿着对称轴方向扩展这些模式，在对称位置填充相应的颜色值，形成完整的对称图案。"
HINTS[171]="这是一个边框绘制的任务，输入是全0的背景矩阵，输出是在相同尺寸的矩阵上绘制颜色为8的单像素边框，保持内部区域为0。"
HINTS[170]="这是一个模式提取任务，输入的特征是底部有一个小的彩色方块区域，输出的特征是该区域的简化版本，你需要识别这个彩色区域的位置，然后提取其对角线上的元素到输出网格中，非对角线位置填充为0。"
HINTS[169]="这是一个连通区域标记和重映射任务，输入图中只有0和5两种颜色，你需要识别所有颜色为5的连通区域，然后按照连通区域的大小（从大到小）分别映射为颜色1、2、3，大小相同的区域保持相同映射关系。"
HINTS[168]="这是一个图案扩展任务，输入和输出的特征是包含稀疏的非零颜色像素在零背景上，你需要识别2x2区域中至少包含2个非零像素的密集区域，然后沿着对角线方向将颜色扩展到相邻的零像素位置。"
HINTS[166]="这是一个空洞填充任务，输入是由颜色8组成的形状和颜色0的背景，你需要识别被颜色8完全包围的封闭区域（空洞），然后将这些区域中的颜色0填充为颜色2。"
HINTS[167]="这是一个模式选择任务，输入和输出都是3x3网格，你需要计算输入中不同颜色的数量并对3取模，然后根据模值选择对角线模式：模0使用主对角线，模1使用第一行，模2使用反对角线，将对应位置的单元格设为5，其他设为0。"
HINTS[165]="这是一个图案填充任务，输入图中包含两种非零颜色组成的特定形状（如菱形），你需要识别该形状的中心垂直线，然后将该线上特定区域（从形状底部向上）填充为第二种颜色。"
HINTS[164]="这是一个镜像复制的任务，输入和输出的特征是3x3网格水平扩展为3x6网格，你需要将输入矩阵的每一行与其水平镜像版本进行连接，形成输出矩阵。"
HINTS[163]="这是一个基于定位点的区块复制任务，输入网格中有一个颜色为4的像素作为定位点，你需要找到这个定位点，然后根据其坐标计算源区块和目标区块的位置，将输入网格中4x4源区块的前3行数据复制到输出网格的对应目标位置，其他区域只保留颜色5的像素，其余都设为0。"
HINTS[162]="这是一个模式填充任务，输入是由一种主要颜色和背景色0组成的网格，你需要识别特定的凹陷模式（通常是L形或U形的边界缺口），然后在凹陷处用颜色1填充3×1的垂直条状区域。"
HINTS[161]="这是一个模式识别和填充的任务，输入是包含多种颜色的网格，输出需要识别出现频率最高的颜色，然后找到该颜色出现在首尾位置的行或列，将这些行或列全部填充为该颜色，其他位置保持为0。"
HINTS[160]="这是一个模式检测任务，输入和输出的特征是识别不完整的3x3矩形（8个位置为1，中心为0），然后在矩形的四个边中点位置（上下左右）添加颜色2的标记，形成十字形图案。"
HINTS[159]="这是一个图案缩放任务，输入包含一个边框（颜色2）和一个内部图案（其他颜色），你需要识别图案的边界框，计算缩放因子，然后在输出中创建一个放大版本的图案，保持边框和内部图案的相对位置和颜色关系。"
HINTS[158]="这是一个模式扩散任务，输入和输出的特征是有一个主导背景色和少量其他颜色的区域，你需要识别输入中特定的3x3颜色模式块，然后在网格中找到所有可以应用该模式的位置，用模式中的颜色替换相应区域的颜色，包括旋转模式进行匹配。"
HINTS[157]="这是一个图形重组任务，输入中有颜色5的连通区域，输出中需要将这些区域移动到网格顶部区域（前几行）并改为颜色1，保持其他部分不变，且移动后的区域必须放置在顶部可用空白（颜色0）位置而不重叠。"
HINTS[155]="这是一个矩阵翻转任务，输入和输出的特征是尺寸相同且颜色分布一致，你需要识别输入矩阵的结构，然后执行上下翻转操作将矩阵的行顺序完全反转。"
HINTS[156]="这是一个矩形内部填充任务，输入和输出的特征是包含由颜色4组成的矩形区域，你需要识别所有由颜色4形成的矩形边界，然后在每个矩形内部填充一个更小的矩形，交替使用颜色1和2进行填充。"
HINTS[154]="这是一个对称变换的任务，输入和输出的特征是有颜色2组成的矩形框结构和颜色5的外部点，你需要识别颜色2矩形框的边界，然后将颜色5的点对称地移动到矩形框内部。"
HINTS[153]="这是一个形状压缩任务，输入是10x10网格包含两个不同颜色的连通区域，输出是3x3网格，需要识别两个区域的位置和形状特征，将左上方的区域压缩到输出的左上方，将右下方的区域压缩到输出的右下方。"
HINTS[152]="这是一个对称扩展任务，输入3x3网格通过水平和垂直镜像操作扩展到6x6网格，需要将输入网格在水平和垂直方向上进行镜像对称复制"
HINTS[151]="这是一个线条检测和填充的任务，输入中通常包含由相同颜色像素组成的垂直线条，你需要识别这些线条的中点或交叉点，然后以该点为中心创建一个3x3的颜色4填充区域（保持中心点原色不变"
HINTS[149]="这是一个区域检测任务，输入是一个11x11网格包含颜色0、6、8，其中8形成两条水平分割线，你需要将网格划分为9个3x3区域，检测哪些区域包含足够多的非零元素（主要是颜色6），然后输出一个3x3网格用1标记这些区域，用0标记其他区域。"
HINTS[150]="这是一个矩阵变换任务，输入和输出的特征是尺寸相同且颜色分布一致，你需要识别输入矩阵的列顺序，然后执行按列反转的操作（即第一列变为最后一列，第二列变为倒数第二列，以此类推"
HINTS[148]="这是一个路径填充任务，输入中有2颜色组成的边界和零散的8颜色点，需要识别边界的方向关系，从每个8颜色点开始向特定方向（由边界位置差决定）填充8颜色区域，并将原来的8颜色点改为4颜色。"
HINTS[147]="这是一个邻域检测任务，输入和输出的特征是相同的网格尺寸，你需要识别颜色为3的单元格，检查其上下左右四个方向是否有相邻的颜色3单元格，如果有则将当前单元格的颜色从3改为8，否则保持颜色3不变。"
HINTS[146]="这是一个网格提取任务，输入是9×3的网格，输出是3×3的网格，你需要识别输入网格中的特定3×3子区域（通常是前、中或后三行），然后提取该子区域作为输出。"
HINTS[145]="这是一个连通区域分析任务，输入是由颜色0和2组成的网格结构，其中颜色2形成路径边界。你需要识别由颜色2边界划分的连通区域，用颜色1填充最大的区域（主要通道），用颜色8标记最小的区域（端点/拐角），其他区域保持原色。"
HINTS[143]="这是一个形状模式识别的任务，输入和输出的特征是识别重复出现的相同形状模式，当检测到某种颜色的形状与之前出现过的形状模式相同时，将该形状的所有像素颜色改为5。"
HINTS[144]="这是一个网格比较任务，输入是一个9x4网格，其中第5行是颜色4的分隔行，将网格分成上下两个4x4区域。你需要比较这两个区域对应位置的值，如果任意一个值为0（背景色），则在输出4x4网格的对应位置输出3，否则输出0。"
HINTS[141]="这是一个对角线扩展任务，输入是一个包含单个非零像素点的网格，你需要识别该点的位置和颜色，然后在输出中生成两条经过该点的对角线（主对角线和反对角线），将对角线上的所有像素设置为输入点的颜色。"
HINTS[140]="这是一个矩阵旋转任务，输入和输出的特征是一个3x3网格且颜色不变，你需要将输入矩阵旋转180度（即上下翻转后再左右翻转）。"
HINTS[142]="这是一个对称扩展的任务，输入是一个3x3的网格，输出是一个6x6的网格，你需要将输入网格在垂直和水平两个方向上进行镜像反射，即先垂直翻转并连接（生成6行），然后对每一行水平翻转并连接（生成6列），从而得到一个中心对称的输出。"
HINTS[139]="这是一个区域填充任务，输入是由颜色4形成的多边形区域和背景0，你需要识别颜色4形成的特定形状（如L形或矩形），然后将其内部的0填充为7，但仅限于特定的3×3子区域。"
HINTS[138]="这是一个边界填充裁剪任务，输入网格有多个同色的边界行列将网格分割，你需要先裁剪外部背景，然后识别边界行列，向内填充边界颜色，最后输出缩小后的核心区域网格。"
HINTS[137]="这是一个嵌套矩形生成任务，输入是包含三个同色点的稀疏网格，输出需要以这些点为基础生成多层嵌套的矩形结构，最内层矩形围绕这些点，外层矩形依次向外扩展形成同心嵌套模式"
HINTS[136]="这是一个对角线延伸任务，输入网格中包含颜色1和2的2×2方块，你需要识别每个颜色方块的位置，然后从该位置开始沿对角线方向（颜色1向右下，颜色2向左上）延伸相应的颜色，直到网格边界。"
HINTS[135]="这是一个区域提取任务，输入是9x9网格，输出是3x3网格，你需要提取输入矩阵中前3行、最后3列的区域作为输出。"
HINTS[134]="这是一个模式检测任务，输入图中包含在3x3网格中规则排列的特定图案对象，你需要识别这些图案的颜色和位置，然后输出一个3x3矩阵表示每个网格位置是否有该图案存在（用图案颜色表示存在，0表示不存在）。"
HINTS[133]="这是一个形状扩展任务，输入包含各种颜色的几何形状，你需要识别特定的连通区域结构，然后按照几何规则将这些形状扩展到周围的空白区域，形成更大或更复杂的图案。"
HINTS[132]="这是一个矩形填充任务，输入中有少量不同颜色的种子点（通常是两个相同颜色的点），你需要识别每组相同颜色的点对，以它们为对角点确定矩形区域，并将该区域填充为对应的颜色。"
HINTS[131]="这是一个滑动块任务，输入包含颜色2的锚点块和颜色3的移动块，你需要识别这些块的相对位置关系，然后让颜色3的块沿着特定方向滑动，并在它们原来的位置轨迹上填充颜色8作为痕迹，同时保持颜色2的锚点块位置不变。"
HINTS[130]="这是一个3x3下采样任务，输入是9x9网格，输出是3x3网格。你需要将输入划分为9个3x3块，对于每个块，移除所有颜色5的像素，然后取剩余颜色中的最大值作为该块的输出值。如果块中只有颜色5，则输出0。"
HINTS[129]="这是一个统计频率的任务，输入是3x3的彩色网格，输出也是3x3的网格，你需要识别输入网格中出现频率最高的颜色，然后将整个输出网格填充为该颜色。"
HINTS[127]="这是一个区域填充任务，输入和输出的特征是由颜色5的垂直线分隔的多个区域，你需要识别中间行中每个区域内的标记颜色（非零非5的颜色），然后将对应区域的所有单元格填充为标记颜色值加5的新颜色。"
HINTS[128]="这是一个垂直移动的任务，输入和输出的特征是包含多个彩色矩形块分布在网格中，你需要识别每个列中的彩色像素块，然后将它们沿着列方向移动到该列的最远可用位置（顶部或底部），保持块的形状和颜色不变"
HINTS[125]="这是一个矩形边框检测和内部填充的任务，输入是由颜色6组成的矩形区域嵌入在颜色8的背景中，输出需要为每个矩形添加颜色3的外边框，并将矩形内部（非边界）的颜色6像素转换为颜色4，同时保持矩形边界为颜色6。"
HINTS[126]="这是一个检测和标记的任务，输入图中包含由相同非零颜色组成的L形或角状结构，你需要识别这些结构的底部拐角位置，然后在输出图的最后一行对应列位置用颜色4进行标记。"
HINTS[124]="这是一个模式延续任务，输入是一个包含重复模式片段的高度小于10的网格，输出需要将输入的模式识别出来并延续扩展到10x10的固定尺寸网格中，保持原有的模式规律。"
HINTS[123]="这是一个矩阵扩展任务，输入是5×5矩阵，输出是10×10矩阵，你需要识别输入矩阵的右下角特征，然后使用max(r,c)模运算规则将输入矩阵平铺复制到输出网格中。"
HINTS[122]="这是一个移动图案的任务，输入和输出的特征是包含由颜色2和3组成的十字形状，你需要识别这个十字图案的中心位置，然后将整个十字图案沿着特定方向移动固定距离。"
HINTS[120]="这是一个矩形区域内部填充任务，输入包含多个不同颜色的矩形区域，你需要识别所有由非0颜色组成的矩形区域的内部区域（去除最外层边框），然后将这些内部区域的像素值改为8。"
HINTS[119]="这是一个对称反射任务，输入包含背景(0)、边界/结构(2)和特殊点(8)，需要识别特殊点的位置，然后在对称方向上创建颜色3的反射图案，形成对称的直线或对角线结构。"
HINTS[118]="这是一个基于距离的染色任务，输入图中包含颜色2作为标记点，你需要识别从每个颜色2像素出发在四个方向上特定距离内的颜色5像素，然后将这些符合条件的颜色5像素改为颜色8。"
HINTS[117]="这是一个中心对称补全任务，输入包含一个不完整的中心对称图案，你需要识别图案的对称中心点，然后将输入图案补全为关于该中心点完全中心对称的输出图案"
HINTS[116]="这是一个垂直镜像拼接的任务，输入和输出的特征是从3x4扩展到6x4，你需要将输入矩阵进行垂直翻转（[::-1]），然后与原矩阵进行垂直拼接（+）。"
HINTS[121]="这是一个模式生成任务，输入是一个13x13网格其中有一个颜色8的像素，输出是一个3x3的十字形网格，其颜色由颜色8所在位置的邻居颜色决定（具体是取上方三个像素的最大值）。"
HINTS[115]="这是一个颜色提取和排序的任务，输入是大尺寸的彩色网格，输出是小尺寸的向量，你需要识别输入中所有不同的颜色，然后按照从上到下的空间出现顺序将这些颜色排列成一个列向量输出。"
HINTS[114]="这是一个网格扩展任务，输入和输出的特征是小网格扩展为大网格，你需要识别输入网格，然后在四周添加边框，边框四角为黑色(0)，上下边缘复制首末行元素，左右边缘复制首末列元素。"
HINTS[113]="这是一个对称镜像任务，输入矩阵顶部有连续的非零行，底部大部分为0，你需要识别顶部的非零行区域，然后在矩阵底部添加这些非零行的逆序副本，保持中间部分为0。"
HINTS[112]="这是一个对称扩展任务，输入包含由颜色2构成的图案和颜色3标记的对称中心，你需要识别对称中心（3标记的2x2方块），然后根据对称中心将颜色2的图案进行镜像对称复制到另一侧。"
HINTS[111]="这是一个位置提取任务，输入和输出的特征是从10×10网格中识别颜色为5的唯一单元格位置，然后以该位置为左上角提取3×3的子网格作为输出。"
HINTS[109]="这是一个对称映射任务，输入图有一个中心分隔行将图分为上下对称部分，你需要移除中心行，将上半部分的模式与下半部分进行中心对称映射，并将特定的输入颜色转换为对应的输出颜色。"
HINTS[110]="这是一个模式补全任务，输入和输出的特征是包含周期性重复模式的网格，其中部分区域被0值覆盖，你需要识别网格的最小行周期和列周期，然后用周期性模式填充所有的0值空白区域。"
HINTS[107]="这是一个图像缩放和模式添加的任务，输入是5x5的网格，输出是输入尺寸的f倍放大网格（f由输入决定），需要将输入图像按比例放大，并在放大后的图像中添加两条交叉的对角线（颜色2），对角线的位置和宽度由缩放因子f和输入图像中特定位置的像素值决定。"
HINTS[108]="这是一个放大和复制任务，输入是稀疏分布的颜色点，输出尺寸翻倍，需要识别所有非零颜色点，为每个点在对应位置创建一个4x4的相同颜色方块。"
HINTS[105]="这是一个区域填充任务，输入是由颜色1形成的边界和颜色0的背景，你需要识别颜色1形成的闭合或半闭合区域，然后将这些区域内部的颜色0填充为颜色2，而保持边界和外部的颜色不变。"
HINTS[104]="这是一个网格扩展和模式复制的任务，输入是3x3网格，输出是9x9网格，你需要识别输入中颜色为2的特殊标记位置，然后在输出中对应的两个对角象限放置4x4的3色块，其他区域填充0。"
HINTS[106]="这是一个矩阵扩展任务，输入是一个2x2或3x3矩阵，输出是一个尺寸翻倍（4x4或6x6）的中心对称矩阵。你需要将输入矩阵与顺时针旋转90度后的矩阵水平拼接形成上半部分，然后将上半部分旋转180度得到下半部分。"
HINTS[103]="这是一个对称性检测任务，输入是3x3网格，主要包含颜色0和2，需要检测网格是否关于水平中心对称（即第一行等于第三行），如果对称则输出1，不对称则输出7"
HINTS[101]="这是一个模式复制和缩放任务，输入中需要识别由1色组成的连通区域模式，以及由2色标记的锚点位置，将1色模式按一定比例（1-3倍）缩放后复制到所有2色锚点位置，并用1色填充模式区域。"
HINTS[102]="这是一个形状填充任务，输入由颜色0（背景）和颜色5（轮廓）组成，你需要识别由颜色5形成的完全闭合矩形区域，然后将这些矩形内部的0填充为颜色2"
HINTS[99]="这是一个形状扩展任务，输入中包含由颜色1组成的闭合形状和内部颜色>1的特殊点，你需要识别这些特殊点，然后从每个特殊点向上、下、左、右四个方向延伸形成十字形状，延伸范围受限于网格边界和颜色1的形状边界。"
HINTS[100]="这是一个数字识别任务，输入包含两个空心矩形数字形状，你需要识别这些数字，选择其中一个数字（基于某种规则如最大数字或最完整形状），然后输出该数字颜色填充的2x2方块。"
HINTS[98]="这是一个矩形框提取任务，输入图中包含一个或多个实心矩形区域，你需要识别这些矩形区域的边界，将每个实心矩形转换为只保留最外层边框的空心矩形，内部区域填充为背景色0。"
HINTS[97]="这是一个孤立点检测和移除的任务，输入和输出的特征是非零颜色点在网格中分布，你需要识别那些周围3x3邻域内颜色值总和不超过自身值的孤立点，然后将这些孤立点移除（设为0），保留其他点不变。"
HINTS[96]="这是一个图像重构任务，输入包含多个不同颜色的图案分散在背景中，你需要识别所有非背景颜色的图案块，通过平移操作将所有图案聚集到中心区域，然后裁剪掉多余的背景，输出一个紧凑的对称布局。"
HINTS[94]="这是一个模式识别和填充的任务，输入中包含由颜色1组成的矩形框结构，你需要识别这些矩形框的位置，然后在每个矩形框的中心列位置创建一条垂直的颜色6的线贯穿整个网格。"
HINTS[93]="这是一个对称转换任务，输入包含一个由颜色5组成的水平分界线将网格分为上下两部分，你需要识别分界线下方区域中的非零颜色点，然后根据水平对称轴将这些点映射到分界线上方的对应位置，并将这些位置的颜色设置为5。"
HINTS[95]="这是一个生成包围块的任务，输入是稀疏颜色5点的网格，输出中每个颜色5点周围生成一个3x3的颜色1块（中心保留5），多个块重叠时重叠区域保持为1。"
HINTS[92]="这是一个路径填充任务，输入和输出的特征是稀疏分布的同色点，你需要识别每个颜色的所有出现位置，然后从每个颜色的第一个点向其他同色点沿直线路径填充该颜色。"
HINTS[91]="这是一个裁剪任务，输入图中包含颜色5（关键点）和颜色8（辅助点），你需要找到所有颜色5的最小边界框，然后在行方向上向上扩展1行、向下扩展2行，列方向不扩展，裁剪出这个区域作为输出。"
HINTS[90]="这是一个寻找最小和矩形区域的任务，输入是包含0、1、5等颜色的网格，你需要找到至少2x2大小的矩形区域中所有单元格和最小的区域，然后将该区域填充为颜色6。"
HINTS[89]="这是一个图案复制与扩展的任务，你需要识别输入中的非单像素连通区域（图案），根据图案的颜色属性决定镜像方向（水平或垂直），然后将这些图案复制到网格的其他空白区域，同时可能添加新的颜色元素来扩展图案"
HINTS[88]="这是一个边界框提取任务，输入中有多种非零颜色，你需要识别出现次数最少的非零颜色（不包括背景色0），然后提取该颜色元素形成的边界框区域作为输出。"
HINTS[87]="这是一个网格旋转任务，输入和输出的尺寸相同且颜色分布不变，你需要将输入网格旋转180度（即上下翻转然后左右翻转）。"
HINTS[86]="这是一个矩形框扩展任务，输入包含一个由两种颜色组成的嵌套矩形结构，你需要识别矩形框的边界，然后将整个矩形结构向外扩展一层，保持内层颜色区域不变但扩大其尺寸，同时在外层添加新的边框。"
HINTS[85]="这是一个网格行模式处理任务，输入和输出的尺寸相同，你需要识别所有连续三行完全相同的行组，然后将每个这样的行组的中间行进行变换：每隔一个非零元素置0（背景色），保持背景0不变。"
HINTS[84]="这是一个矩阵模式转换任务，输入矩阵的第一列有特定颜色且其余为0，你需要保持第一列不变，在对角线位置（从右上到左下）添加颜色2，并在最后一行（除第一列外）添加颜色4。"
HINTS[82]="这是一个像素扩散任务，输入图中第一行有少量非零像素，需要将这些非零像素按照对角线方向向下和向右扩散，生成棋盘格模式的输出，保持原始非零像素的位置不变同时在其周围生成相同颜色的新像素。"
HINTS[83]="这是一个对称扩展任务，输入和输出的特征是3x4网格到6x8网格的尺寸翻倍转换，你需要识别输入矩阵的模式，然后在水平和垂直两个方向上进行镜像对称复制操作。"
HINTS[81]="这是一个角点检测的任务，输入包含颜色8组成的L形结构，你需要识别2x2窗口中包含3个8和1个0的L形内角位置，然后将这些0填充为颜色1。"
HINTS[80]="这是一个基于网格的颜色传播任务，输入是由特定边界颜色分隔的网格结构，你需要识别网格的交点位置，然后根据相邻块的颜色信息，将网格中的某些区域填充为特定的颜色，实现颜色的对称传播。"
HINTS[79]="这是一个模式识别任务，输入是14x14的网格，输出是3x3的网格，你需要识别输入网格中最频繁出现的3x3子模式，然后输出这个最常见的3x3模式。"
HINTS[78]="这是一个重力模拟任务，输入中有颜色1的区域和颜色2的物体，你需要识别颜色2的像素，让它们向上移动直到遇到颜色1的区域或边界为止。"
HINTS[77]="这是一个区域填充任务，输入和输出的特征是包含颜色2的像素，你需要识别颜色2的像素及其周围的连通区域（使用曼哈顿距离<3的邻域），然后将这些区域中与2相邻的非2像素替换为颜色4"
HINTS[76]="这是一个模板匹配和扩展的任务，输入中包含颜色为4的条形结构，你需要识别这些结构的末端或特定位置，然后在匹配的位置上复制或扩展模板，生成新的颜色3和1的点。"
HINTS[75]="这是一个模式复制的任务，输入和输出的特征是输入图有一个垂直的5列和左上角3x3彩色块，你需要识别输入中标记点（颜色1）的位置，然后在每个标记点对应的位置复制左上角的3x3块。"
HINTS[74]="这是一个对称最小化任务，输入和输出的特征是30x30网格，你需要识别关于中心对称的位置对，然后选择每对位置中颜色值较小的那个，特别要注意颜色9会被其他较小的颜色值替换。"
HINTS[73]="这是一个映射标记的任务，输入网格的第3行包含标记点（颜色1），第5行是底部边界（颜色5），你需要将第3行的标记点映射到第5行的对应位置（将5改为1），同时清除第3行的所有标记点（改为0），其他行保持不变。"
HINTS[72]="这是一个矩阵比较任务，输入是一个13x5的矩阵，其中第7行是颜色为4的分隔行，你需要将前6行与后6行（跳过分隔行）进行逐行逐列比较，如果对应位置的值不同则输出3，相同则输出0，最终生成6x5的输出矩阵。"
HINTS[71]="这是一个图形规整化任务，输入和输出的特征是保持相同尺寸但颜色分布更规整，你需要识别输入中不规则的色块区域，然后通过对称检测和颜色替换操作，将这些区域转换为与周围主要颜色一致或形成更对称的模式。"
HINTS[70]="这是一个区域转换任务，输入包含颜色0、1、8组成的网格，其中颜色8标记了一个矩形区域，你需要识别该区域的范围，并将该区域内所有颜色为1的像素转换为颜色3。"
HINTS[69]="这是一个模板匹配和复制的任务，输入图中包含彩色图案和由8组成的框架，你需要识别彩色图案的形状和相对位置，然后在图中找到形状相同的8框架，将彩色图案复制到这些框架位置，同时移除所有的8。"
HINTS[68]="这是一个定位和绘制的任务，输入和输出的特征是网格中散布着各种颜色的点，你需要识别出现频率最低的颜色（如有多个取第一个），然后以该颜色位置为中心绘制一个3x3的方块，外围用颜色2填充，中心保持原颜色。"
HINTS[67]="这是一个裁剪任务，输入是一个m x n的矩阵（m行n列，且n>=m），输出是一个m x m的矩阵，由输入矩阵每一行的前m个元素组成。"
HINTS[66]="这是一个区域填充任务，输入中有两个颜色3的起始点和两个颜色2的标记点，你需要识别由8包围且内部为0的矩形区域，然后用颜色3填充该区域形成连接路径。"
HINTS[65]="这是一个子矩阵提取任务，输入是一个具有中心十字对称结构的奇数尺寸矩阵，你需要识别矩阵中的特殊颜色（非对称轴颜色），然后提取包含该特殊颜色的象限子矩阵，输出尺寸为(输入尺寸-1)/2。"
HINTS[64]="这是一个特殊点扩展任务，输入和输出的特征是识别出现频率较低的特殊颜色点，然后根据这些点的位置，将它们沿着水平或垂直方向扩展到最近的边界或其他特殊点，填充相同的颜色。"
HINTS[63]="这是一个区域填充任务，输入和输出的特征是矩阵外围边界保持不变，内部被非0值包围的连续0区域需要被识别出来，然后将这些区域中的0填充为颜色3"
HINTS[62]="这是一个对称填充任务，输入的特征是包含一个颜色2的标记点和若干其他颜色的色块，输出的特征是以标记点为对称中心将色块对称复制到另一侧形成完整对称图案，同时将背景色从0改为3。"
HINTS[61]="这是一个填充空白区域的任务，输入网格中包含用0表示的空白区域，输出需要识别网格中已有的数值模式（通常是数学序列或周期性模式），然后用正确的数值填充所有空白区域，保持网格的整体模式一致性。"
HINTS[60]="这是一个行变换任务，输入和输出的尺寸相同，你需要识别每行中第一个和最后一个非零元素，然后将该行转换为：左侧颜色重复5次，中间插入颜色5，右侧颜色重复5次，全零行保持不变。"
HINTS[59]="这是一个区块填充任务，输入和输出的特征是网格被两条水平线分成三个区域，每个区域包含3x3的区块，你需要识别包含最多非5颜色像素的3x3区块，然后将该区块的所有像素填充为该颜色。"
HINTS[58]="这是一个螺旋路径生成任务，输入是全0矩阵，输出需要创建一个从矩阵边界开始、顺时针向内旋转的螺旋路径，将路径上的单元格填充为颜色3，其他单元格保持为0。"
HINTS[57]="这是一个形状提取和重排任务，输入是稀疏的8x8矩阵包含少量非零像素形成的形状，输出是3x6矩阵，需要提取所有非零像素并按照某种顺序（可能是行优先或列优先）重新排列到输出网格中。"
HINTS[56]="这是一个模式识别任务，输入是3x3网格，包含一种主颜色和背景色0，输出是1x1网格，需要识别主颜色在网格中的分布模式（如十字形、四角形、加号形等），然后根据模式类型输出对应的颜色编码。"
HINTS[55]="这是一个区域划分和填充的任务，输入中有颜色8形成的水平和垂直分隔线，你需要识别这些分隔线划分出的不同区域，然后根据每个区域相对于分隔线的位置（左上、右上、左下、右下等）填充对应的颜色值。"
HINTS[53]="这是一个平移任务，输入和输出的特征都是3x3网格，你需要识别输入中的所有非零元素（1或2），然后将它们整体向下平移一行，超出边界的元素消失，底部空出的位置用0填充。"
HINTS[54]="这是一个图案扩散任务，输入图中包含背景色和特殊点，你需要识别孤立的特殊点颜色，然后以这些点为中心向水平和垂直方向扩散该颜色，形成十字形图案，并在对角线交叉点处理特殊的颜色组合模式。"
HINTS[51]="这是一个路径延伸填充任务，输入和输出的特征是存在一个出现次数最少的特殊颜色点，你需要识别这个特殊点，然后根据其周围非零颜色的方向（上下左右）进行延伸，将路径上的0填充为该特殊点的颜色值。"
HINTS[52]="这是一个行检测任务，输入和输出的特征都是3x3网格，你需要识别输入矩阵中所有元素相同的行，将这些行输出为颜色5，其他行输出为颜色0。"
HINTS[50]="这是一个连接点的任务，输入和输出的特征是包含颜色8的标记点，你需要识别每行和每列中颜色8的位置，然后在同一行或同一列的两个8点之间（不包括端点）用颜色3填充连接。"
HINTS[49]="这是一个形状提取和重排的任务，输入包含多个彩色形状（非0颜色）和背景（0），输出需要选择某个特定颜色的形状，将其所有像素重新排列成一个紧凑的矩形，保持该颜色的总像素数量不变。"
HINTS[48]="这是一个连通性检测任务，输入网格包含颜色0、2、8，你需要从第一个颜色2开始进行区域生长，如果能清除所有颜色2（所有颜色2连通），则输出0；如果还有孤立的颜色2剩余，则输出8。"
HINTS[47]="这是一个方向线填充任务，输入中有两个特殊点（颜色8和7），你需要识别这两个点的位置，然后从8点所在列填充为8，从7点所在行填充为7，在行列交点处使用颜色2，其他位置保持为0。"
HINTS[46]="这是一个基于分隔符的序列重组任务，输入特征是3行矩阵中包含颜色5作为分隔符，你需要识别颜色5的位置将矩阵分割成多个块，然后对这些颜色块进行重组和压缩，生成输出矩阵。"
HINTS[45]="这是一个行填充任务，输入和输出的特征是当一行的首尾元素相同且非零时，将该行所有元素填充为该颜色，否则保持原行不变。"
HINTS[44]="这是一个形状匹配和区域交换的任务，输入和输出的特征是包含多个颜色区域，你需要识别颜色不为0或5的连通区域，找到与之形状相同且被颜色5完全包围的空白区域（颜色0），然后将这两个区域的内容进行交换。"
HINTS[43]="这是一个模式传播任务，输入的特征是第0行有颜色5的特殊点，输出的特征是将这些特殊点所在的列中所有其他行（除第0行外）的颜色5替换为颜色2，保持第0行不变"
HINTS[42]="这是一个检测和镜像填充的任务，输入中包含颜色3的点形成的对角线结构，你需要识别这些对角线的起点和长度，然后在相对于原点的对称位置绘制颜色8的矩形区域。"
HINTS[41]="这是一个对称形状扩展任务，输入和输出的特征是包含对称分布的颜色点，你需要识别输入中对称分布的颜色点模式，然后将这些点扩展连接成更完整的对称形状（如菱形、三角形等）。"
HINTS[40]="这是一个颜色替换任务，输入和输出的特征是上下边界有不同的颜色且中间有颜色3的点，你需要识别颜色3的位置，根据其相对于网格中心线的位置将其替换为上边界颜色或下边界颜色。"
HINTS[39]="这是一个区域提取任务，输入是10×10的稀疏图案，输出是3×3的核心区域，你需要识别图案中非零元素聚集的有效区域，然后提取包含主要图案特征的3×3区块。"
HINTS[38]="这是一个模式计数任务，输入是一个9×9的网格包含0、1、2三种颜色，输出是一个1×5的二进制向量，表示网格中相邻1对（水平相邻的两个1）的总数量，用二进制形式表示（高位在前，补零到5位）。"
HINTS[37]="这是一个颜色传播任务，输入中有几个非零颜色的点，每个颜色有固定的延伸方向（如2、6、9向右下，3、8向左下等），需要从每个颜色的起始位置开始，沿着其指定的对角线方向延伸，直到网格边界，保持背景为0。"
HINTS[36]="这是一个区域提取任务，输入是一个大网格，其中包含由特定颜色值组成的密集区域，输出是该区域的最小边界框，但尺寸可能比实际图案稍大以保持完整性，周围用0填充。"
HINTS[35]="这是一个投影映射任务，输入和输出的特征是包含一个由8组成的矩形区域和边缘的孤立颜色点，你需要识别8区域的边界位置，然后将边缘的孤立颜色点投影到8区域的对应边界位置。"
HINTS[34]="这是一个对角线扩展任务，输入图中包含颜色2的种子点和另一种非零颜色，你需要识别颜色2的位置，根据其相邻单元格确定扩展方向，然后沿着该对角线方向将路径上的所有单元格填充为另一种非零颜色。"
HINTS[32]="这是一个重力沉降任务，输入和输出的特征是网格尺寸相同且颜色分布不变，你需要识别每列中的非零元素，然后让它们受到重力作用向下沉降到底部，保持原有的相对顺序。"
HINTS[33]="这是一个图案投影任务，输入包含一个十字形骨架和分散的图案，你需要识别每个6x6区块内的非骨架图案，然后在对应的背景位置上用骨架颜色进行填充投影。"
HINTS[30]="这是一个物体移动的任务，输入和输出的特征是有多个分散的颜色块或点，你需要识别所有非零的颜色元素，然后将它们垂直移动到网格的中间行区域，保持它们的水平位置不变。"
HINTS[31]="这是一个裁剪任务，输入和输出的特征是输入图中包含由非零颜色组成的形状，周围有大量0背景，你需要识别非零像素构成的有效区域，然后执行去除所有全零行和列的操作，提取出包含形状的最小边界框。"
HINTS[29]="这是一个区域提取任务，输入矩阵中包含由相同颜色像素组成的水平分隔条带，你需要识别这些条带形成的边界，然后提取上下条带之间的中间区域作为输出。"
HINTS[28]="这是一个模式生成任务，输入的特征是几乎全为0的背景上有两个不同颜色的孤立像素点，输出的特征是根据这两个点的颜色生成分层边框结构：上半部分使用第一个点的颜色，下半部分使用第二个点的颜色，每部分都有全色行和边框行交替的固定模式。"
HINTS[26]="这是一个对称比较任务，输入矩阵中间有一列颜色1作为分隔符，将矩阵分为左右对称的两部分，你需要比较每行左右对称位置的像素值，当两个位置都是0时输出8，否则输出0，从而生成一个压缩后的输出矩阵。"
HINTS[27]="这是一个凹陷填充任务，输入是由颜色0和1组成的网格，其中颜色1形成各种形状，你需要识别颜色1形状左侧的凹陷或凹槽区域，然后将这些凹陷区域中的0填充为颜色2。"
HINTS[24]="这是一个线条扩展任务，输入和输出的特征是输入图中的非零颜色点作为锚点，需要识别这些点的位置和颜色，然后将每个颜色扩展为完整的水平或垂直线条，其他区域保持背景色。"
HINTS[25]="这是一个异常点移除任务，输入和输出的特征是识别并移除那些与周围主要模式不匹配的孤立颜色点，这些点通常是单个像素且颜色值与周围区域明显不同，需要检测这些异常点并将其颜色改为0或移除。"
HINTS[22]="这是一个邻域提取任务，输入是11x11网格，输出是3x3网格，你需要识别输入中特定的颜色5（黄色）锚点，然后以该锚点为中心提取其3x3邻域作为输出。"
HINTS[23]="这是一个形状识别和重新着色的任务，输入图中包含颜色5形成的各种几何形状，你需要识别这些形状中的L形或角形结构，然后将这些结构的拐角部分着色为2，其他部分着色为8，同时保持背景颜色0不变。"
HINTS[21]="这是一个网格分割计数任务，输入网格被特定颜色的分隔线分割成多个矩形区域，你需要识别分隔线的位置，计算被分割的区域行列数量，然后输出相应尺寸的单一颜色网格"
HINTS[20]="这是一个对称补全任务，输入包含部分对称图案的非零点，需要识别图案的中心位置，然后通过90度旋转对称操作，在对称位置复制相应的颜色值来补全整个对称图案。"
HINTS[18]="这是一个物体重新定位任务，输入中包含多个离散的彩色形状（连通组件），你需要识别这些形状，然后将它们移动到网格的边界区域（通常是角落或边缘），保持形状的结构不变但改变其位置。"
HINTS[19]="这是一个图形放大和边缘增强的任务，输入的小图形需要被放大2倍，然后在原始非零像素周围的0像素位置用颜色8填充，形成图形的轮廓和背景网格"
HINTS[17]="这是一个模式恢复任务，输入网格中包含周期性重复的模式但被0值中断，你需要识别最小的重复单元模式，然后用这个模式填充所有的0值区域来恢复完整的周期性网格。"
HINTS[16]="这是一个颜色映射任务，输入和输出的特征都是3x3的网格且尺寸相同，每行颜色模式重复。你需要识别输入中的每个颜色值，然后通过固定的映射表（0→0,1→5,2→6,3→4,4→3,5→1,6→2,7→7,8→9,9→8）将其转换为对应的输出颜色。"
HINTS[15]="这是一个像素扩展任务，输入包含特定标记像素（颜色1和2），需要为每个标记像素在其对角线（颜色2→颜色4）或十字方向（颜色1→颜色7）生成相应的扩展像素，其他像素保持不变。"
HINTS[14]="这是一个目标提取任务，输入矩阵被分为上下两部分，上半部分是干扰背景，下半部分包含一个特定颜色的图案，你需要识别出除了0之外出现频率最低的颜色，然后提取该颜色图案的最小外接矩形区域作为输出。"
HINTS[13]="这是一个模式生成任务，输入是稀疏的二维网格，其中包含两个非零颜色的种子点，输出需要生成相同尺寸的网格，其中按照种子点的位置关系创建周期性交替的颜色条纹模式。"
HINTS[12]="这是一个模式扩展任务，输入图中包含由两种颜色组成的十字形结构（中心颜色和周围颜色），你需要识别这些十字形，然后将它们扩展成完整的菱形/星形模式，保持中心颜色不变并在对角线方向复制周围颜色。"
HINTS[11]="这是一个区域染色任务，输入被颜色5的十字分割为9个3x3区域，你需要识别每个区域中不包含颜色8的3x3块，然后以该块左上角3x3区域中的某个颜色值为基准，将对应的整个3x3区域统一染成该颜色，同时保持颜色5的十字结构不变。"
HINTS[10]="这是一个路径分割任务，输入是由颜色5形成的连通路径，输出需要将路径分割成不同的段，从右向左（或根据位置关系）依次用颜色1、2、3、4等标记路径的不同部分。"
HINTS[9]="这是一个网格传播任务，输入是由主要颜色构成的网格框架和少量彩色块，你需要识别每个彩色块的位置和颜色，然后根据其行列索引模3的结果决定传播方向（水平或垂直），将彩色块的颜色传播到同一行或同一列的对应位置，直到遇到相同颜色的边界为止。"
HINTS[8]="这是一个对象移动的任务，输入和输出的特征是有颜色2的对象和颜色8的目标位置，你需要识别颜色2的对象和颜色8的目标位置，然后将颜色2的对象移动到颜色8的目标位置。"
HINTS[7]="这是一个模式填充任务，输入的特征是包含一个3种颜色序列的对角线模式，输出的特征是用这个颜色序列以对角线循环方式填充整个7x7网格，你需要识别输入中的3种颜色及其顺序，然后按照(i+j) mod 3的索引来循环填充这些颜色。"
HINTS[6]="这是一个模式匹配任务，输入是3x7网格，中间第4列是分隔符颜色5，你需要识别每行左右两侧（各3列）相同位置是否同时为1，如果是则在3x3输出的对应位置输出2，否则输出0。"
HINTS[4]="这是一个形状平移任务，输入和输出的特征是包含由非零颜色组成的形状，你需要识别这些形状，然后将每个形状整体向右移动一列，保持形状的完整性。"
HINTS[5]="这是一个模式复制任务，输入和输出的特征是包含特定的形状模式（通常是3x3的密集区域），你需要识别输入中最密集的3x3区域作为模板，然后按照固定的步长（如4的倍数）在网格的其他位置复制这个模板，可能伴随颜色的变换。"
HINTS[3]="这是一个矩阵扩展任务，输入是6×3的0-1矩阵，输出是9×3的0-2矩阵，你需要将所有的1转换为2，然后根据输入矩阵的模式重复特定的行来扩展矩阵到9行。"
HINTS[2]="这是一个区域填充任务，输入由背景色0和边界色3组成，输出需要识别被颜色3完全包围的封闭区域，并将这些区域内的0填充为颜色4。"
HINTS[1]="这是一个矩阵扩展任务，输入是3x3网格，输出是9x9网格，你需要将每个输入元素扩展为3x3的输出块，扩展规则取决于该输入元素的值和位置，通常是非零元素在输出中保持原值并按特定模式排列，零元素在输出中保持为零或形成特定背景模式。"

In [ ]:
plt.figure(figsize=(3, 1), dpi=150)
plt.imshow([list(range(10))], cmap=cmap, norm=norm)
plt.xticks(list(range(10)))
plt.yticks([])
plt.tick_params(axis='x', color='r', length=0, grid_color='none')

plt.show()

In [ ]:
for TASK_ID in range(1,401):
    # Load the task data
    try:
        task_data = load_task(TASK_ID)
        print(f"TASK{int(TASK_ID):03d}:")
        print(HINTS[TASK_ID])
        #print(f"Loaded task {TASK_ID:03d} with {len(task_data['train'])} training examples and {len(task_data['test'])} test examples and {len(task_data['arc-gen'])} arc-gen examples.")
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find the file task{TASK_ID:03d}.json. Please place it in the current directory.")
        
    # Visualize training and test examples
    print("Training examples:")
    for idx, example in enumerate(task_data['train']):
        fig, axs = plt.subplots(1, 2, figsize=(4, 2))
        plot_grid(example['input'], ax=axs[0], title=f"Train {idx} – input")
        plot_grid(example['output'], ax=axs[1], title=f"Train {idx} – output")
        plt.show()
    
    print("Test examples:")
    for idx, example in enumerate(task_data['test']):
        fig, ax = plt.subplots(1, 1, figsize=(2, 2))
        plot_grid(example['input'], ax=ax, title=f"Test {idx} – input")
        plt.show()